In [1]:
import os
os.chdir('..')
!pwd

/Users/krishna/Documents/Research/Simultaneous Classifier/folktables-experiments/multidecomp


In [2]:
import bilevel
from bilevel.Adahedge import *
from bilevel.build_all_models import *
from bilevel.OnlineLinearExpert import *
from bilevel.OnlineTreeExpert import *

In [3]:
import pandas as pd
import numpy as np

# Original Adult ordering, undropped sensitive groups

## Online ridge baselines

In [4]:
df_og_oh = pd.read_pickle('./data_small/dataframes/df_adultog_oh.pkl')

In [5]:
X_dat_og_oh = df_og_oh.drop('income', axis=1)
y_dat_og_oh = df_og_oh['income']

In [6]:
A_t_og_oh = np.load('./data_small/nparrays/A_t_adultog.npy')

In [7]:
dirname = './models_adult/baseline/'
filename = 'ridge_og_oh'

In [8]:
expridge = OnlineLinearExpert(X_dat_og_oh, y_dat_og_oh, l2_pen = 0.0)
b_ridgebase = build_baseline_alwayson(dirname, filename, A_t_og_oh, expridge)

100%|██████████| 49531/49531 [00:15<00:00, 3177.14it/s]


In [9]:
A_t_og_oh.shape

(49531, 8)

In [10]:
for gnum in range(A_t_og_oh.shape[1]):
  print(b_ridgebase.expert.cumloss_groupwise[gnum][-1])

12.508590287453616
61.14885755769352
107.20875707048076
9.91389446268583
1375.8878453850696
348.50439068844014
1218.1635540749396
1566.6679447633792


## Anh with online ridge meta experts

In [10]:
dirname = './models_adult/Anh/'
filename = 'ridgemeta_og_oh'

In [11]:
# experts = [OnlineLinearExpert(X_dat_og_oh, y_dat_og_oh, l2_pen = 1.0) for _ in range(A_t_og_oh.shape[1])]
experts = [OnlineLinearExpert(X_dat_og_oh, y_dat_og_oh, l2_pen = 0.0) for _ in range(A_t_og_oh.shape[1])]

In [12]:
b_Anh = build_Anh(dirname, filename, A_t_og_oh, experts)

100%|██████████| 49531/49531 [00:50<00:00, 980.16it/s] 


In [13]:
for gnum in range(8):
  print(b_Anh.Anh.cumloss_groupwise_ada[gnum][-1], len(b_Anh.Anh.cumloss_groupwise_ada[gnum]))

12.805537953588432 471
61.490646376534755 1519
107.45616697332657 4694
10.167420056158432 406
1376.3677603361211 42441
348.8098958200054 16357
1219.4776358757206 33174
1568.2875316957266 49531


In [13]:
for gnum in range(8):
  print(b_Anh.Anh.cuml_loss_curve[gnum][-1], len(b_Anh.Anh.cuml_loss_curve[gnum]))

18.653817880741098 471
81.41566809873368 1519
153.26694468195157 4694
15.660961471871213 406
2017.5993049821036 42441
535.0489399658729 16357
1751.547757149526 33174
2286.5966971154044 49531


## Online hoeffding tree baseline

In [7]:
df_le_og = pd.read_pickle('./data_small/dataframes/df_adultog_le.pkl')

In [9]:
df_le_og

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.00000,0.0,4.0,9.0,0.800000,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.00000,0.0,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.00000,0.0,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.00000,0.0,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.00000,0.0,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49526,0.653061,0.246575,0.00000,0.0,4.0,9.0,0.800000,2.0,0.0,4.0,1.0,41.0,0.850649,5.0
49527,0.775510,0.273973,0.03137,0.0,6.0,9.0,0.800000,2.0,0.0,1.0,1.0,40.0,0.340668,12.0
49528,0.551020,0.095890,0.00000,0.0,4.0,8.0,0.666667,4.0,1.0,4.0,1.0,39.0,0.129230,12.0
49529,0.397959,0.095890,0.00000,0.0,4.0,15.0,0.600000,4.0,1.0,4.0,0.0,39.0,0.149091,1.0


In [13]:
A_t_og = np.load('./data_small/nparrays/A_t_adultog.npy')

In [8]:
y_dat_le_og = df_le_og['income']
X_dat_le_og = df_le_og.drop('income', axis=1, inplace=False)

In [10]:
X_dict_le_og = X_dat_le_og.to_dict(orient='records')

In [11]:
exphoefftree = OnlineHoeffdingTree(X_dict_le_og, y_dat_le_og)

In [12]:
dirname = './models_adult/baseline/'
filename = 'hoefftree_og_oh'

In [14]:
b_htreebase = build_baseline_alwayson(dirname, filename, A_t_og, exphoefftree) #implementable

100%|██████████| 49531/49531 [00:04<00:00, 10200.39it/s]


In [15]:
for gnum in range(8):
  print(b_htreebase.expert.cumloss_groupwise[gnum][-1])

15.600329066893071
71.72805562180785
134.1976627126758
14.164809710784594
1709.5784307728145
505.5597665935454
1439.709521291429
1945.2692878849784


## Anh with hoeff tree meta expert

In [18]:
dirname = './models_adult/Anh/'
filename = 'hoefftree_meta_ogoh'

In [22]:
treeexperts = [OnlineHoeffdingTree(X_dict_le_og, y_dat_le_og) for _ in range(A_t_og.shape[1])]

In [23]:
b_Anh = build_Anh(dirname, filename, A_t_og, treeexperts)

100%|██████████| 49531/49531 [00:23<00:00, 2106.21it/s]


In [24]:
for gnum in range(8):
  print(b_Anh.Anh.cuml_loss_curve[gnum][-1])

16.26539573133961
74.45884210220974
129.93536467784554
14.594519310554261
1711.1432186997329
430.1824956925233
1516.2148448291566
1946.3973405216666
